In [1]:
import time
import numpy as np
import random
import copy, sys

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

from scipy import stats
from numpy import linalg as LA

In [2]:
#Function to load the data file to memeory.

#Input: File path to read.
#Output: A 2d numpy array with all loaded samples from the file to read in string.

def parseFile_raw(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0
    
    with open(file) as txtfile:
        for row in txtfile:
            
            row = row.split(',')
            row[-1] = row[-1].strip()
            count += 1
            if count != 1:
                content.append(row[:5])
            #content.append([row[21]] + row[0:4] + [row[22]] + [row[32]] + [row[29]] + [row[6]] \
            #               + [row[-2]] + [row[-1]] + [row[12].strip("'")])
          
            #if count == 1000:
                #break

    content_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')

    return content_mat


#Function to filter the samples with no missing values. 
#Input: mat - 2d Numpy Array.
#Onput: mat - 2d Numpy Array with all samples that have no Missing values.

def filter_full_feature(mat):
    row_count = 0
    full_list = []
    for row in mat:
        if 'N/A' in row or 'NA' in row:
            pass
        else:
            full_list.append(row_count)

        row_count += 1
    print('There are a total of ' + str(len(full_list)) + ' samples fed into the model')
    mat = mat[full_list, :]
    return mat

In [48]:
def cf(mat, sim_user, feat, k):
    mat += 0.01
    sim_user += 0.01
    n = sim_user.shape[0]
    sim_user = np.concatenate((np.zeros((n,feat)), sim_user), axis=1)
    cos_sim = sim_user.dot(mat.T) / LA.norm(sim_user) / LA.norm(mat,axis = 1)
    
    idx = np.flip( np.argsort(cos_sim) , 1)[:,:k]
    tmp = np.flip( np.sort(cos_sim), 1)[:,:k]
    
    print(mat[idx,:][0].shape)
    print(stats.mode(mat[idx,:][0], axis = 0)[0][0][:3])
    print(sim_user.shape)
    sim_user[0,:3] = stats.mode(mat[idx,:][0], axis = 0)[0][0][:3]
    print(sim_user)
#     for i in range(feat):
#         print("val: ", mat[idx,i])
#         print(mat[idx,i])
#         print(stats.mode(mat[idx,i], axis = 1)[0].T[0])
#         sim_user[:,i] = np.sum(mat[idx,i] * tmp, axis = 1)
#         sim_user[:, i] = stats.mode(mat[idx,i], axis = 1)[0].T[0]
        
    sim_user = (sim_user - 0.01).astype(int)
    return sim_user

In [49]:
def main():
    mat = filter_full_feature(parseFile_raw('test_small.csv'))
    mat = mat.astype(float)
    print(mat.shape)
    sim_user = np.array([[8,8,1]]).astype(float)
    feat, k = 2, 10
    
    print(cf(mat,sim_user,feat,k)) #,want 2 new features, k nearest users
    
    
if __name__ == "__main__":    
    main()
    

Reading data is complete! Running time is 0.0005006790161132812s!
There are a total of 64 samples fed into the model
(64, 5)
(10, 5)
[ 1.01 -0.99  7.01]
(1, 5)
[[ 1.01 -0.99  7.01  8.01  1.01]]
[[ 1 -1  7  8  1]]
